In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation # animaciones
from IPython.display import HTML

from scipy.sparse import diags, kron, csr_matrix, csc_matrix, identity, eye
from scipy.sparse.linalg import spsolve, norm, inv, eigsh

import time

# Colisiones Kink-Antikink en ecuación $\phi_4$

En el presente jupyter se simulará una colisión de ondas kink-antikink en la ecuación $\phi_4$ dada por
$$
\begin{equation}
\left.
\begin{array}[c]{rll}
 \phi_{tt} - \phi_{xx} - \phi + \phi^3= & 0 & \text{sobre }\Omega=\left(0,T\right]\times\left(-L,L\right)\\
\phi(0,x)= & \phi_K(x+x_0) + \phi_{\bar{K}}(x-x_0)-1 & \text{en }x\in [-L, L]\\
\phi_t(0,x)= & -v\phi'_K(x+x_0) - v\phi'_{\bar{K}}(x-x_0) & \text{en }x\in [-L, L]
\end{array}
\right\} \quad (1)
\end{equation}
$$

Con condiciones de borde periódica o Neumann según lo que se requiera. Los objetivos son:
* Encontrar el mejor método de simulación, es decir, aquel método con menor propagación de errores.
* Obtener la velocidad de salida después de la colisión.
* Obtener los gráficos correspondientes, de forma de replicar los resultados del paper.

## introducción

Los métodos utilizados para simular la ecuación corresponden a diferencias finitas (esquemas explícito e implícito) y métodos espectrales. Es esperable que la presición de la simulación mejor con respecto a cada método usado, donde los esquemas espectrales tengan menor dispersión de error y deseablemente deban ser utilizados en la obtención de las velocidades de salida.

Se definen en primer lugar funciones con las condiciones iniciales del problema:

In [2]:
# Función secante hiperbolica
def sech(x):
    return 1/np.cosh(x)

# Función kink
def phi_k(x, v):
    return np.tanh(x/(np.sqrt(2) * np.sqrt(1-v**2)))

# Derivada kink
def phi_k_t(x, v):
    return sech(x/(np.sqrt(2) * np.sqrt(1-v**2)))**2

# Condición inicial
def phi_0(x, x0, v):
    return phi_k(x+x0, v) - phi_k(x-x0, v)-1

# Condición inicial derivada temporal
def phi_1(x, x0, v):
    return -v/np.sqrt(2*(1-v**2)) * (phi_k_t(x+x0, v) + phi_k_t(x-x0, v))

En todas las simulaciones, se utilizarán $x_0 = 10$, $L = 20$ y tiempo final $T=100$. Este último parámetro podría variar según la velocidad de cálculo computacional de cada esquema.

In [3]:
x0 = 10
xi = -20
xf = 20
tf = 100

A modo de testeo, se probarán las velocidades $v \in \{0.35, 0.186, 0.1988, 0.213\}$.

In [4]:
# Velocidades
vs = np.arange(0.19, 0.2, 0.001) #[0.189, 0.1988, 0.2384, 0.35]

## Diferencias finitas: Esquema explícito

Para $h>0$ y $dt = h/2$ (se debe cumplir la condición CFL) se obtienen una grilla espacial de $N$ puntos y una temporal de $N_T = T/dt$ puntos, además del siguiente problema discretizado centralizado en tiempo y espacio:

$$
\frac{\phi_j^{n+1} - 2\phi_j^n + \phi_j^{n-1}}{dt^2} - \frac{\phi_{j+1}^n - 2\phi_j^n + \phi_{j-1}^n}{h^2} + f(\phi_j^n) = 0
$$

donde $f(\phi) = -\phi + \phi^3$. Definimos $\lambda = \frac{dt}{h}$ y despejando $\phi_j^{n+1}$ se obtiene un esquema explícito para la ecuación $\phi_4$:

$$
\phi_j^{n+1} = 2\left (1-\lambda^2\right )\phi_j^n - \phi_j^{n-1} + \lambda^2\phi_{j+1}^n + \lambda^2\phi_{j-1}^n - dt^2f(\phi_j^n)
$$
Con condiciones iniciales:

* $\phi_j^0 = \phi_K(x_j+x_0) + \phi_{\bar{K}}(x_j-x_0)-1$
* $\phi_j^1 = \left(-v\phi'_K(x_j+x_0) - v\phi'_{\bar{K}}(x_j-x_0)\right) dt + \phi_j^0$

y con condiciones de borde periódicas (que será resuelto utilizando el método ``np.roll``). Este esquema se implementa a continuación en la función ``solve_explicit``.

In [5]:
def solve_explicit(x0, xi, xf, h, dt, tf, v):
    
    # Puntos de las grillas
    Nx = int((xf - xi)/h)
    Nt = int(tf/dt)

    # Crea grillas
    x = np.arange(xi,xf, h)
    t = np.arange(0, tf, dt)

    # Vector para guardar solución
    phi = np.zeros((Nt, Nx))

    # Setea condiciones iniciales
    phi[0] = phi_0(x, x0, v)
    phi[1] = phi_1(x, x0, v) * dt + phi[0]
    
    # Lambda
    l = dt/h
    
    # Itera puntos temporales
    for n in range(2, Nt):
        
        # Instante anterior y paso actual
        phi_j_last = phi[n-2]
        phi_j = phi[n-1]

        # Condiciones periódicas
        
        # Vecino derecha
        phi_j_der = np.roll(phi_j, shift=1)

        # Vecino izquierda
        phi_j_izq = np.roll(phi_j, shift=-1)

        # Solución instante actual
        phi_j_sgte =  2 * (1 - l**2) * phi_j - phi_j_last + l**2 * (phi_j_der + phi_j_izq) + dt**2 * (phi_j - phi_j**3)

        # Guarda solución
        phi[n] = phi_j_sgte
    
    # Retorna grillas y solución
    return x, t, phi